In [2]:
from azureml.core import Experiment, Workspace
from azureml.train.automl.constants import Scenarios
from azureml._restclient.jasmine_client import JasmineClient
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException


compute_name = "cpu-clusters"
ws = Workspace.from_config()

if compute_name not in ws.compute_targets: # WSに存在しなければ新規作成
    compute_config = AmlCompute.provisioning_configuration(
        vm_size = "Standard_F4S_V2", 
        max_nodes=4, 
        idle_seconds_before_scaledown = 300
    )
    compute_target = ComputeTarget.create(ws, compute_name, compute_config)
    compute_target.wait_for_completion(show_output=True)
compute_target = ComputeTarget(workspace=ws, name=compute_name)

推論環境もトレーニング時の環境と同環境でなければ、エラーを起こすため、
`JasmineClient`を使用して、トレーニング時の環境を取得する

In [ ]:
training_experiment = Experiment(ws, 'classification_titanic')
 
jasmine_client = JasmineClient(
    service_context=training_experiment.workspace.service_context,
    experiment_name=training_experiment.name,
    experiment_id=training_experiment.id,
)
 
training_env = jasmine_client.get_curated_environment(
    scenario=Scenarios.AUTOML,
    enable_dnn=False,
    enable_gpu=False,
    compute=compute_target,
    compute_sku=compute_target.vm_size
)

# 抽出したトレーニング環境を .yml としてエクスポートする
training_env.save_to_directory('./env', overwrite=True)